In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# from ipynb.fs.full.Preprocessing import clean_data
# %load_ext tensorboard

# df = pd.read_csv('./Data/songs_cleaned.csv', index_col='Unnamed: 0')
# print(df.shape)
# df.describe()

In [4]:
df = pd.read_csv('../../songs_cleaned.csv', index_col='Unnamed: 0')

In [6]:
# Simple autoencoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

h1 = Dense(32, activation = 'relu')(input_song)

encoded = Dense(encoded_dim, activation = 'relu')(h1)

dh1 = Dense(32, activation='sigmoid')(encoded)

decoded = Dense(df.shape[1], activation='sigmoid')(dh1)

autoencoder = Model(input_song, decoded)

encoder = Model(input_song, encoded)

autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [7]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

Train on 469075 samples, validate on 117269 samples
Epoch 1/100
469075/469075 [==============================] - 23s 49us/sample - loss: 0.3371 - val_loss: 0.3316
Epoch 2/100
469075/469075 [==============================] - 23s 48us/sample - loss: 0.3011 - val_loss: 0.3211
Epoch 3/100
469075/469075 [==============================] - 23s 50us/sample - loss: 0.2959 - val_loss: 0.3181
Epoch 4/100
469075/469075 [==============================] - 24s 51us/sample - loss: 0.2946 - val_loss: 0.3174
Epoch 5/100
469075/469075 [==============================] - 23s 48us/sample - loss: 0.2940 - val_loss: 0.3169
Epoch 6/100
469075/469075 [==============================] - 23s 49us/sample - loss: 0.2938 - val_loss: 0.3168
Epoch 7/100
469075/469075 [==============================] - 24s 51us/sample - loss: 0.2937 - val_loss: 0.3167
Epoch 8/100
469075/469075 [==============================] - 24s 51us/sample - loss: 0.2936 - val_loss: 0.3165


### Shallow NN Summary:
0 hidden layers: Val Loss: 0.3160
1 hidden layer: 

In [8]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

In [12]:
print(embeddings_df.shape)
embeddings_df

(586344, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3.470329,1.184762,-0.0,3.144354,1.041142,2.506661,2.963388,1.308488,1.876889,1.609339,5.813118,4.160755,2.801373,6.369508,4.479137,2.048349
1,2.950846,0.692970,-0.0,1.260800,2.372062,1.424065,1.766102,3.973808,1.751205,1.503640,4.932892,4.102685,1.840484,4.968472,4.601698,2.108422
2,4.472703,6.916587,-0.0,3.051311,4.255427,4.078246,6.610500,5.424572,5.226394,2.885791,4.807835,8.209723,6.890685,2.668237,4.231436,3.645994
3,0.533942,1.481426,-0.0,4.247745,1.795213,3.724570,6.466702,2.883682,2.688429,2.568929,3.822564,2.750004,4.210648,3.195352,3.263384,3.226213
4,2.028276,4.151319,-0.0,2.583655,2.820859,2.327025,2.896990,3.921444,3.261746,1.292208,6.060862,3.102732,4.927852,3.230224,1.068353,2.544864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,4.507507,4.525337,-0.0,1.797992,4.408144,3.942313,3.207572,3.014376,2.259826,3.485083,7.321132,4.829626,4.268849,4.802366,3.543452,2.489765
586668,4.967984,3.997606,-0.0,2.682946,5.384268,4.684574,2.361519,3.863641,2.298209,4.028028,7.587534,5.696170,5.347732,3.508698,5.025582,3.510531
586669,2.513383,4.620409,-0.0,3.026764,4.275283,6.560318,5.074978,4.096501,2.595194,1.959881,3.780759,4.040783,5.324980,2.569675,2.156150,2.363571
586670,4.446574,4.300193,-0.0,4.233162,4.264293,5.235272,1.241963,4.270548,4.267136,2.311171,6.389156,5.381336,5.270690,4.331300,3.387321,5.655649


In [11]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)

NearestNeighbors(algorithm='ball_tree', n_neighbors=11)

In [15]:
len(encoded_songs[0])

16

In [ ]:
# Saving the encoded_songs embedding as a df and the Nearest Neighbours model for use in our app
import pickle

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

# Saving the embeddings to the disk
embeddings_df.to_csv('./saved_models/embeddings_df_001.csv')

# save the model to disk
filename = './saved_models/nearestneigh_001.sav'
pickle.dump(nn, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [16]:
embeddings_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3.470329,1.184762,-0.0,3.144354,1.041142,2.506661,2.963388,1.308488,1.876889,1.609339,5.813118,4.160755,2.801373,6.369508,4.479137,2.048349
1,2.950846,0.692970,-0.0,1.260800,2.372062,1.424065,1.766102,3.973808,1.751205,1.503640,4.932892,4.102685,1.840484,4.968472,4.601698,2.108422
2,4.472703,6.916587,-0.0,3.051311,4.255427,4.078246,6.610500,5.424572,5.226394,2.885791,4.807835,8.209723,6.890685,2.668237,4.231436,3.645994
3,0.533942,1.481426,-0.0,4.247745,1.795213,3.724570,6.466702,2.883682,2.688429,2.568929,3.822564,2.750004,4.210648,3.195352,3.263384,3.226213
4,2.028276,4.151319,-0.0,2.583655,2.820859,2.327025,2.896990,3.921444,3.261746,1.292208,6.060862,3.102732,4.927852,3.230224,1.068353,2.544864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,4.507507,4.525337,-0.0,1.797992,4.408144,3.942313,3.207572,3.014376,2.259826,3.485083,7.321132,4.829626,4.268849,4.802366,3.543452,2.489765
586668,4.967984,3.997606,-0.0,2.682946,5.384268,4.684574,2.361519,3.863641,2.298209,4.028028,7.587534,5.696170,5.347732,3.508698,5.025582,3.510531
586669,2.513383,4.620409,-0.0,3.026764,4.275283,6.560318,5.074978,4.096501,2.595194,1.959881,3.780759,4.040783,5.324980,2.569675,2.156150,2.363571
586670,4.446574,4.300193,-0.0,4.233162,4.264293,5.235272,1.241963,4.270548,4.267136,2.311171,6.389156,5.381336,5.270690,4.331300,3.387321,5.655649


In [ ]:
# testing with Beautiful Day - U2

raw_df = pd.read_csv('./Data/tracks.csv')
test_songs = {'Beautiful Day': '0gzqZ9d1jIKo9psEIthwXe'}
for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == '0gzqZ9d1jIKo9psEIthwXe'][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])

    score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]

    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')


In [ ]:
# Testing our model with multiple songs
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes


In [ ]:
# Deep variational encoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

encoded_h1 = Dense(64, activation = 'relu')(input_song)
encoded_h2 = Dense(32, activation = 'relu')(encoded_h1)
encoded = Dense(encoded_dim, activation = 'relu')(encoded_h2)

decoded_h1 = Dense(32, activation='relu')(encoded)
decoded_h2 = Dense(64, activation='relu')(decoded_h1)
decoded = Dense(df.shape[1], activation='sigmoid')(decoded_h2)

deep_autoencoder = Model(input_song, decoded) # techincally autoencoder_and_decoder

deep_encoder = Model(input_song, encoded) # Half of the autoencoder

deep_autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [ ]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

deep_autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

### Deep NN Summary
Val loss = 0.2960

In [ ]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)

In [ ]:
encoded_songs.shape

In [ ]:
# # Testing
# raw_df = pd.read_csv('./Data/tracks.csv')

# test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
#               'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# # Loop over all our test songs and get the recommendations
# recommendations = {}

# print('DeepSpot\'s recommendation system:\n\n')

# for name, id in test_songs.items():
#     query = raw_df.index[raw_df['id'] == id][0]
#     test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
#     score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
#     k_closest_songs_indexes = close_image_idx[0][1:]
    
#     print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
#     for i in range(len(k_closest_songs_indexes)):
#         song = raw_df.iloc[k_closest_songs_indexes[i], :]
#         print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][i]))
#         print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
#     print('___________________________________________________________________________________\n')
    
#     recommendations[name] = k_closest_songs_indexes



# Training a Variational Autoencoder

In [ ]:
# Variational Autoencoder
from keras import layers

class Sampling(layers.Layer):
    """
    Uses (z_mean, z_log_var) to sample a z vector from the latent N-dimensional Normal distribution
    i.e. Z is the vector encoding a digit.
    """
    def call(self, inputs):
        
        # recall from the video that z_mean and z_log_var are vectors 
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        
        # sample from an N-dimensional normal distribution
        # epsilon is given shape (batch, dim) because we are adding it to z_mean which has shape (batch, dim)
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        
        # output of tf.exp() is a vector 
        simga = tf.exp(0.5 * z_log_var)
        
        # this is our hideen latent vector made up of a mean and variance vector 
        # variance vector is scaled by epsilon, which is sampled from a normal distribtuion 
        # the the video guy said "stocastic" in reference to epsilon, he meant random 
        Z = z_mean + simga * epsilon
        
        # return hidden latent vector 
        return Z

In [ ]:
# recall from the video that the more dimensions that our latent vector has
# the better the results of our model 
latent_dim = 16

# shape of our input data
# we are creating our input layer using Keras's Input() class
# the only thing that input layers really do is define the dimensionality of the input data for the model
encoder_inputs = Input(shape=(df.shape[1], ))

# these are the hidden layers

# pass data vector into FCFF layer 
encoded = Dense(encoded_dim, activation = 'relu')(encoder_inputs)

# recall that ordinarly the output of the last encoding layer is the latent vector 
# but here we are creating two output layers for our encoder - one for the mean and one for the log variance 
# returns a 2-dim mean vector
z_mean = Dense(latent_dim, name="z_mean")(encoded)
# returns a 2-dim log variance vecotr 
z_log_var = Dense(latent_dim, name="z_log_var")(encoded)
# pass mean and variance vector into Sampling class in order to create the Z vector,  Z = mean + var * epsilon
z = Sampling()([z_mean, z_log_var])

# ok, now let's put it all together 
# this is our encoder model 
# inputs are the original images
# outputs are the Z vectors: mean, log variance, and the complete Z, i.e. vector Z = mean + var * epsilon
encoder = Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
# the input layer to our decoder has the same dimensionality as the latent vector
# because the latent vector is the input to the decoder model
latent_inputs = Input(shape=(latent_dim,))

# these are the hidden layers of our decoder 
# the data at this point is in a vector, the Z latent vector 

# this is the final layer in out decoder
# therefore this layer outputs the reconstruction of the original image 
decoder_outputs = Dense(df.shape[1], activation='sigmoid')(latent_inputs)

# this is our decoder model 
decoder = Model(inputs=latent_inputs, outputs=decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
from tensorflow import keras

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        """
        This class build a Variational Auto-Encoder. It accepts an Encoder and Decoder model as input. 
        
        Note
        ----
        This VAE class is inheriting Keras's Model API so that it can use the Model class methods 

        """
        # how python 3 handels inheritance 
        super(VAE, self).__init__(**kwargs)
        # set encoder model as class attribute
        self.encoder = encoder
        # set decoder model as class attribute 
        self.decoder = decoder
        # set mean function as class attribute - this calculates the total loss
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        # set mean function as class attribute - this calculates the reconstruction loss
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        # set mean function as class attribute - this calculates the kl loss
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        """
        Returns all loses in a list
        """
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        """
        Training our model via gradient descent and back-propagation 
        """
        
        # we used tf.GradientTape() in Sprint 2 Module 2 to run Gradient Descent from scratch 
        with tf.GradientTape() as tape:
            # pass input data into encoder model 
            # output of encoder model is the hidden state distribution parameters and hidden state vector 
            z_mean, z_log_var, z = self.encoder(data)
            
            # pass hidden state vector into decoder model 
            reconstruction = self.decoder(z)
            
            # calculate the reconstruction loss 
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.MeanSquaredError(data, reconstruction)#, axis=(1, 2)
                )
            )
            
            # calculate the kl loss
            #                (1 + z_simga   - (z_mean)^2        - e^(z_simga) ) 
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            # recall that we used tf.reduce_sum() in Sprint 2 Module 4 assignment 
            # it takes the sum of the vector components 
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            # calculate the total loss by adding the two loss components 
            total_loss = reconstruction_loss + kl_loss
            
        # now that we have calculated the loss function, we can perform Gradient Descent
        # we are passing in the loss function and the weights that we want to update via Gradeint Descent 
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # log the total loss
        self.total_loss_tracker.update_state(total_loss)
        
        # log the reconsgrution loss 
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        
        # log the kl loss 
        self.kl_loss_tracker.update_state(kl_loss)
        
        # return all the losses in a dictionary 
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
# instantiate a Variational Auto-Encoder model 
vae = VAE(encoder, decoder)

# complie the model 
vae.compile(optimizer=keras.optimizers.Nadam())

# train the model weights 
vae.fit(df, epochs=10, workers=10)
# if you have access to multiple processors, set parameter `workers` to N - 1
# where N is the total number of processors that you have 

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
autoencoder.save('saved_model/autoencoder_002')
encoder.save('saved_model/encoder_002')